In [ ]:
import ta

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import USD, BTC, ETH, LTC
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.agents import DQNAgent



In [ ]:
# FETCH HISTORICAL DATA

cdd = CryptoDataDownload()

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
], axis=1)

bitstamp_data


In [3]:
# DEFINE EXCHANGES

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC")
)

In [ ]:
# DEFINE ADDITIONAL DATA (TA, etc)

bitstamp_btc = bitstamp_data.loc[:, bitstamp_data.columns.str.startswith("BTC")].copy()

ta.add_all_ta_features(
    bitstamp_btc,
    colprefix='BTC:',
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_steams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    
feed = DataFeed(bitstamp_steams)

feed.next()

In [5]:
# PORTFOLIO
portfolio = Portfolio(USD, [
    Wallet(bitstamp, 1 * BTC),
    # Wallet(bitstamp, 1 * BTC),
    # ...
])

In [6]:
# ENVIRONMENT

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=True
)

env.observer.feed.next()

{'internal': {'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/BTC:/free': 1.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 1.0,
  'bitstamp:/BTC:/worth': 8740.99,
  'net_worth': 8740.99},
 'external': {'bitstamp:/BTC:date': Timestamp('2018-05-15 06:00:00'),
  'bitstamp:/BTC:unix': 1526364000,
  'bitstamp:/BTC:open': 8733.86,
  'bitstamp:/BTC:high': 8796.68,
  'bitstamp:/BTC:low': 8707.28,
  'bitstamp:/BTC:close': 8740.99,
  'bitstamp:/BTC:volume': 559.93,
  'bitstamp:/BTC:volume_adi': -137.6651163311054,
  'bitstamp:/BTC:volume_obv': 559.93,
  'bitstamp:/BTC:volume_cmf': nan,
  'bitstamp:/BTC:volume_fi': nan,
  'bitstamp:/BTC:volume_em': nan,
  'bitstamp:/BTC:volume_sma_em': nan,
  'bitstamp:/BTC:volume_vpt': nan,
  'bitstamp:/BTC:volume_vwap': nan,
  'bitstamp:/BTC:volume_mfi': nan,
  'bitstamp:/BTC:volume_nvi': 1000.0,
  'bitstamp:/BTC:volatility_bbm': nan,
  'bitstamp:/BTC:volatility_bbh': nan,
  'bitstamp:/BTC:volatility_bbl': nan,
  'bitstamp:/BTC:volatility_bbw': nan,
  

In [7]:
# SETUP AND TRAN DQN AGENT

agent = DQNAgent(env)

agent.train(n_steps=200, n_episodes=2, save_path="agents/")

C:\Users\thiago\AppData\Local\Temp\ipykernel_21376\2065615943.py:3: DeprecationWarning: Call to deprecated class DQNAgent. (Builtin agents are being deprecated in favor of external implementations (ie: Ray)) -- Deprecated since version 1.0.4.
  agent = DQNAgent(env)


TypeError: Error when deserializing class 'Conv1D' using config={'name': 'conv1d_2', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': (4,), 'strides': (2,), 'padding': 'causal', 'data_format': 'channels_last', 'dilation_rate': (1,), 'groups': 1, 'activation': {'module': 'keras.layers', 'class_name': 'PReLU', 'config': {'name': 'p_re_lu_2', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'alpha_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'alpha_regularizer': None, 'alpha_constraint': None, 'shared_axes': None}, 'registered_name': None}, 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'HeUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularize